# 0. Introduction

In this tutorial, we will use Spark Streaming to stream data from a Cloudant database, and process this continously received stream of data using Spark SQL. 

Our processing pipeline goes through three stages:
1. *_changes feed* is streamed from the given Cloudant database using `CloudantReceiver`. `CloudantReceiver` will receive _changes feed of the database, extract individual JSON documents from the feed, and store these documents in Spark's memory for processing by Spark Streaming. 
2. Spark Streaming will break up this continous stream of documents into batches. Each batch is a separate RDD, and in our case represents a set of documents collected within 10 secs window. This sequence of batches, or sequence of RDDs is what is called a discretized stream or DStream. 
3. Each RDD of the DStream is processed using Spark SQL.

```
|                1                  | -> |            2               |    |          3             |
|_changes feed ->... doc3 doc2 doc1 | -> |... [doc4 doc3] [doc2 doc1] | -> |... [pBatch2] [pBatch1] |
|      CloudantReceiver             | -> | Spark Streaming: DStream   | -> |      Spark SQL         |
```

In the steps below, we:
1. Initialize StreamingContext and DStream
2. Define processing of DStream using Spark SQL
3. Actually start processing, and stop it after some time.

# 1. Initializing DStream

## 1.1 Provide the details of your cloudant account in *properties* map:

- *cloudant.host* - the fully qualified account https URL
- *cloudant.username* - the Cloudant account username
- *cloudant.password* - the Cloudant account password
- *database* - database which changes you want to recieve 

In [1]:
import org.apache.spark.streaming.{ Seconds, StreamingContext, Time }
import org.apache.spark.{ SparkContext, SparkConf}
import org.apache.spark.sql.SQLContext
import org.apache.spark.rdd.RDD
import java.util.concurrent.atomic.AtomicLong

import com.cloudant.spark.CloudantReceiver


val properties = Map(
    "cloudant.host"-> "wow1.cloudant.com", 
    "cloudant.username"-> "wow1",
    "cloudant.password"-> "wow1Cloudant",
    "database"-> "election2016"
)

## 1.2 Initialize StreamingContext and DStream

- Initialize a StreamingContext with a 10 seconds batch size
- Create a DStream of database changes using CloudantReceiver

In [2]:
val ssc = new StreamingContext(sc, Seconds(10))
val changesDStream = ssc.receiverStream(new CloudantReceiver(properties))

## 2. Define processing of DStreams

- Get SQLContext
- For every batch:
  - Create a dataframe `tweetsDF`
  - Create `tweetsDF2` dataframe with fields `gender`, `state`, and `polarity` 
  - Calculate and display the cumulative count of tweets
  

In [3]:
val sqlContext = new SQLContext(sc)
import sqlContext.implicits._

val curTotalCount = new AtomicLong(0)
changesDStream.foreachRDD((rdd: RDD[String], time: Time) => {
    println(s"========= $time =========")
    val tweetsDF = sqlContext.read.json(rdd)
    
    if (!tweetsDF.schema.isEmpty) {
        val tweetsDF2 = tweetsDF.select($"cde.author.gender", 
                $"cde.author.location.state",
                $"cde.content.sentiment.polarity")
        tweetsDF2.show(10)
        
        curTotalCount.getAndAdd(tweetsDF2.count())
        println("Current total count:" + curTotalCount)
    }
})

# 3. Start receiving and processing of data

- Start StreamingContext
- Allow processing to run for 120 secs
- Manually stop processing 

All previous instructions were just initilizations and definions, and nothing will happen until we start StreamingContext. After the start, the data will be received and processed. Since, DStream is continous,  it will not stop until we manually stop the processing.

In [4]:
ssc.start()
Thread.sleep(120000L)
ssc.stop(true)

========= 1473435100000 ms =========
+-------+--------+--------+
| gender|   state|polarity|
+-------+--------+--------+
|   male|    Utah| NEUTRAL|
|unknown|    null| NEUTRAL|
|   male|        | NEUTRAL|
|   null| Arizona|    null|
|   male|New York|POSITIVE|
|unknown|    null| NEUTRAL|
|unknown|    Utah| NEUTRAL|
|unknown|        |POSITIVE|
|   null|    null|    null|
|unknown|        | NEUTRAL|
+-------+--------+--------+
only showing top 10 rows

Current total count:300
========= 1473435110000 ms =========
========= 1473435120000 ms =========
========= 1473435130000 ms =========
========= 1473435140000 ms =========
========= 1473435150000 ms =========
========= 1473435160000 ms =========
========= 1473435170000 ms =========
========= 1473435180000 ms =========
+-------+--------+--------+
| gender|   state|polarity|
+-------+--------+--------+
|   male|        | NEUTRAL|
|unknown|    Utah| NEUTRAL|
|unknown|    null| NEUTRAL|
|unknown|    null| NEUTRAL|
|   male|    Utah| NEUTRAL|
|